In [1]:
import sys
sys.path.append('/home/azhuravl/work')

import stereoanyvideo.datasets.video_datasets_custom as video_datasets_custom

In [2]:
import importlib
importlib.reload(video_datasets)

In [3]:
dataset_driving = video_datasets_custom.SequenceSceneFlowDatasetCamera(
    aug_params=None,
    root="/home/azhuravl/scratch/SceneFlow",
    dstype="frames_cleanpass",
    sample_len=59,
    things_test=False,
    add_things=False,
    add_monkaa=False,
    add_driving=True,
    split="test",
    stride=5,
)
len(dataset_driving)



In [4]:
data_0 = dataset_driving[0]


In [5]:
data_0['img'].shape

In [6]:
import torch

sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/15_10_25_depth')

import collect_dataset

frames_tensor, depths, poses_tensor, K_tensor = collect_dataset.extract_video_data(
    data_0,
    # baseline=0.532725,
    # image_size=data_0['metadata'][0][0][1],
    )

In [7]:
sys.path.append('/home/azhuravl/work/TrajectoryCrafter')

import models.utils as utils

warper_old = utils.Warper(device='cuda')

In [8]:
K_tensor.shape

In [9]:
from tqdm import tqdm

warped_images = []
masks = []
warped_depths = []

warped_images_once = []

for i in tqdm(range(10, frames_tensor.shape[0])):
    
    transformation_1 = poses_tensor[i:i+1].clone()
    transformation_2 = poses_tensor[10:11].clone()
    
    warped_frame2, mask2, warped_depth2, flow12 = warper_old.forward_warp(
        frame1=frames_tensor[i:i+1],
        mask1=None,
        depth1=depths[i:i+1],
        transformation1=transformation_1,
        transformation2=transformation_2,
        intrinsic1=K_tensor[0].unsqueeze(0),
        intrinsic2=K_tensor[0].unsqueeze(0),
        mask=False,
        twice=True,
    )
    warped_frame_once, _, _, _ = warper_old.forward_warp(
        frame1=frames_tensor[i:i+1],
        mask1=None,
        depth1=depths[i:i+1],
        transformation1=transformation_1,
        transformation2=transformation_2,
        intrinsic1=K_tensor[0].unsqueeze(0),
        intrinsic2=K_tensor[0].unsqueeze(0),
        mask=False,
        twice=False,
    )
    
    warped_images.append(warped_frame2)
    masks.append(mask2)
    warped_depths.append(warped_depth2)
    
    warped_images_once.append(warped_frame_once)

In [10]:
import matplotlib.pyplot as plt

import numpy as np

# plot warped image j, mask j, warped depth j
j = 0
k = 30

frame = frames_tensor[j+10].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5
warped_image = warped_images[k][0].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5
target_frame = frames_tensor[k+10].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5

plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.imshow(frame)
plt.title('Source Frame')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(warped_image)
plt.title('Warped Image to Frame {}'.format(j))
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(target_frame)
plt.title('Target Frame {}'.format(j))
plt.axis('off')


## Resize and Save

In [16]:
import torch
import torch.nn.functional as F


def get_resize_strategy(original_height, original_width, target_height=384, target_width=672):
    """Determine the best resize strategy based on aspect ratios"""
    original_aspect = original_width / original_height
    target_aspect = target_width / target_height  # 672/384 = 1.75
    
    if original_aspect > target_aspect * 1.2:
        return "crop_width"
    # If original is much taller, crop height  
    elif original_aspect < target_aspect * 0.8:
        return "crop_height"
    else:
        # For remaining cases, just use resize (may cause some distortion)
        return "resize"


def smart_video_resize(video, target_height=384, target_width=672, interpolation_mode='bilinear'):
    """
    Smart resize for videos with various aspect ratios
    
    Args:
        video: tensor of shape [T, C, H, W]
        target_height: target height (default 384)
        target_width: target width (default 672)
        interpolation_mode: 'bilinear' for RGB videos, 'nearest' for masks
        
    Returns:
        video: tensor of shape [T, C, target_height, target_width]
    """
    T, C, H, W = video.shape
    
    strategy = get_resize_strategy(H, W, target_height, target_width)
    
    print(f"resize strategy: '{strategy}' for original size ({H}, {W}), aspect ratio {W/H:.2f}, target ratio {target_width/target_height:.2f}")
    
    if strategy == "resize":
        # Simple resize when aspect ratios are similar or as fallback
        video = F.interpolate(
            video, 
            size=(target_height, target_width), 
            mode=interpolation_mode, 
            align_corners=False
        )
        
    elif strategy == "crop_width":
        # Video is too wide - crop width first, then resize
        target_aspect = target_width / target_height
        new_width = int(H * target_aspect)
        
        # Center crop width
        start_w = (W - new_width) // 2
        video = video[:, :, :, start_w:start_w + new_width]
        
        # Then resize to target
        video = F.interpolate(
            video,
            size=(target_height, target_width),
            mode=interpolation_mode,
            align_corners=False
        )
        
    elif strategy == "crop_height":
        # Video is too tall - crop height first, then resize
        target_aspect = target_width / target_height
        new_height = int(W / target_aspect)
        
        # Center crop height
        start_h = (H - new_height) // 2
        video = video[:, :, start_h:start_h + new_height, :]
        
        # Then resize to target
        video = F.interpolate(
            video,
            size=(target_height, target_width),
            mode=interpolation_mode,
            align_corners=False
        )
    
    return video

In [17]:
frames_tensor.shape, cond_video.shape

In [18]:
from models.utils import save_video
import os
import torch
import matplotlib.pyplot as plt
import matplotlib
import numpy as np


def make_dimensions_even(tensor):
    """Pad tensor to make height and width even numbers"""
    _, h, w, c = tensor.shape
    pad_h = h % 2
    pad_w = w % 2
    
    if pad_h > 0 or pad_w > 0:
        # Pad bottom and right if needed
        tensor = torch.nn.functional.pad(tensor, (0, 0, 0, pad_w, 0, pad_h))
    
    return tensor


def apply_colormap_to_depth(depth_tensor, colormap='viridis', inverse=True):
    """Apply colormap to depth tensor for better visualization"""
    # Create mask for zero values
    zero_mask = (depth_tensor == 0)
    
    if inverse:
        # Compute inverse depth, avoid division by zero
        depth_processed = torch.where(depth_tensor > 0, 1.0 / depth_tensor, torch.zeros_like(depth_tensor))
    else:
        depth_processed = depth_tensor
    
    # Normalize non-zero values to [0, 1]
    if depth_processed[~zero_mask].numel() > 0:
        depth_norm = depth_processed / depth_processed[~zero_mask].max()
    else:
        depth_norm = depth_processed
    
    # Convert to numpy and apply colormap
    depth_np = depth_norm.cpu().numpy()
    colormap_func = matplotlib.colormaps.get_cmap(colormap)
    depth_colored = colormap_func(depth_np)  # Returns RGBA
    
    # Convert back to tensor, drop alpha channel
    depth_colored_tensor = torch.from_numpy(depth_colored[..., :3]).to(depth_tensor.device)
    
    # Set zero depth areas to black
    zero_mask_expanded = zero_mask.unsqueeze(-1).expand_as(depth_colored_tensor)
    depth_colored_tensor[zero_mask_expanded] = 0.0  # Black for zero depth
    
    return depth_colored_tensor



save_dir = '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/driving_resized'
os.makedirs(save_dir, exist_ok=True)


### Save videos

# twice warped video
cond_video = (torch.cat(warped_images) + 1.0) / 2.0  # [T, 3, H, W] in [0,1]
cond_video = smart_video_resize(cond_video)
cond_video_padded = make_dimensions_even(
    cond_video.permute(0, 2, 3, 1)
)
save_video(
    cond_video_padded,
    f'{save_dir}/warped_video_twice.mp4',
    fps=10,
)

# once warped
cond_video_once = (torch.cat(warped_images_once) + 1.0) / 2.0  # [T, 3, H, W] in [0,1]
cond_video_once = smart_video_resize(cond_video_once, target_height=384, target_width=672)
cond_video_once_padded = make_dimensions_even(
    cond_video_once.permute(0, 2, 3, 1)
)
save_video(
    cond_video_once_padded,
    f'{save_dir}/warped_video_once.mp4',
    fps=10,
)

# input and ref videos
frames_tensor_padded = smart_video_resize(frames_tensor)
frames_tensor_padded = make_dimensions_even(
    (frames_tensor_padded.permute(0, 2, 3, 1) + 1.0) / 2.0
)
save_video(
    frames_tensor_padded[10:],
    f'{save_dir}/input_video.mp4',
    fps=10,
)
save_video(
    frames_tensor_padded[:10],
    f'{save_dir}/ref_video.mp4',
    fps=10,
)

### Save depths

# warped depths
warped_depths_tensor = torch.cat(warped_depths)
warped_depths_tensor = smart_video_resize(warped_depths_tensor)
warped_depths_colored = apply_colormap_to_depth(warped_depths_tensor.squeeze(1), inverse=True)
warped_depths_padded = make_dimensions_even(warped_depths_colored)
save_video(
    warped_depths_padded,
    f'{save_dir}/warped_depths.mp4',
    fps=10,
)

# input and ref depths
depths_resized = smart_video_resize(depths)
depths_colored = apply_colormap_to_depth(depths_resized.squeeze(1), inverse=True)
depths_padded = make_dimensions_even(depths_colored)
save_video(
    depths_padded[10:],
    f'{save_dir}/input_depths.mp4',
    fps=10,
)
save_video(
    depths_padded[:10],
    f'{save_dir}/ref_depths.mp4',
    fps=10,
)

# masks
masks_tensor = torch.cat(masks)
masks_tensor = smart_video_resize(masks_tensor)
masks_padded = make_dimensions_even(
    masks_tensor.permute(0, 2, 3, 1).repeat(1, 1, 1, 3)
    )
save_video(
    masks_padded,
    f'{save_dir}/masks.mp4',
    fps=10,
)


In [14]:
import torch, json

# extrinsics
with open(f"{save_dir}/extrinsics.json", "w") as f:
    json.dump(poses_tensor.cpu().tolist(), f, indent=2)

# intrinsics
with open(f"{save_dir}/intrinsics.json", "w") as f:
    json.dump(K_tensor[0].cpu().tolist(), f, indent=2)


In [15]:
import numpy as np

frames_np = ((frames_tensor.cpu().permute(0, 2, 3, 1).numpy() + 1.0) / 2.0).astype(np.float32)

caption = trajcrafter.get_caption(opts, frames_np[opts.video_length // 2])
with open(f"{save_dir}/caption.txt", "w") as f:
    f.write(caption)
print("Caption:", caption)

In [58]:
depths.min(), depths.max(), warped_depths_tensor.min(), warped_depths_tensor.max()

## Save:

### 1,000 for visualization
- 49 input RGB + 10 ref
- 49 warped RGB 2x
- 49 warped RGB 1x
- 49 input depth + 10 ref
- 49 warped depth
- 49 masks
- caption
- camera extr
- camera intr


### 10,000 for training
- 49 input latents
- 10 ref latents
- 49 warped RGB 2x latents
- 49 input depth latents
- 10 ref depth latents
- 49 warped depth latents
- 49 masks
- caption latents

### TODO
- ++ implement saving for visualization, 1 subdir per sample
- ++ crop or interpolate videos to fixed size
- rename resized variables
- encode training data to latents
- implement saving for training, to a tar directly

In [98]:
frames_tensor.shape

In [65]:
# get size of frames_tensor in MB
frames_tensor.element_size() * frames_tensor.nelement() / (1024 ** 2)

# save to /home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/driving/frames.pt
torch.save(
    frames_tensor.cpu(), 
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/driving/frames.pt'
    )

torch.save(
    cond_video_padded.cpu(), 
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/driving/warped.pt'
    )

torch.save(
    depths.cpu(), 
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/driving/depths.pt'
    )

torch.save(
    warped_depths_tensor.cpu(), 
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/driving/warped_depths.pt'
    )

In [63]:
frames_tensor.shape

## Point Cloud Warper

In [ ]:
import sys
sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/06_10_25_vggt')

import warper_point_cloud
warper = warper_point_cloud.GlobalPointCloudWarper(device='cuda')


In [ ]:
from tqdm import tqdm

pc_list = []
color_list = []

with torch.no_grad():
    for j in tqdm(range(frames_tensor.shape[0])):
        i = 0
        points, colors, _ = warper.create_pointcloud_from_image(
            frames_tensor[i:i+1],
            None,
            depths[i:i+1],
            # torch.inverse(poses_tensor[i:i+1]).to('cuda'),
            poses_tensor[i:i+1].to('cuda'),
            # K_tensor[i:i+1],
            K_tensor.unsqueeze(0),
            1,
        )
        pc_list.append(points)
        color_list.append(colors)

In [ ]:
import utils_autoregressive as utils_ar

warped_images = []
masks = []        

for i in tqdm(range(frames_tensor.shape[0])):

    # warped_image, mask = warper.render_pointcloud_zbuffer_vectorized_point_size(
    warped_image, mask = warper.render_pointcloud_zbuffer_vectorized_fixed(
        pc_list[i],
        color_list[i],
        poses_tensor[i:i+1].to('cuda'),
        K_tensor.unsqueeze(0).to('cuda'),
        image_size=frames_tensor.shape[-2:],
        point_size=2,
    )

    
    cleaned_mask = utils_ar.clean_single_mask_simple(
        mask[0],
        kernel_size=9,
        n_erosion_steps=1,
        n_dilation_steps=1
        )
    # should stay in [-1, 1] range
    
    cleaned_mask = cleaned_mask.unsqueeze(0)
    warped_image = warped_image * cleaned_mask
    
    warped_images.append(warped_image)
    masks.append(cleaned_mask)


In [ ]:
import matplotlib.pyplot as plt

import numpy as np

# plot warped image j, mask j, warped depth j
j = 0
k = 30

frame = frames_tensor[j].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5
warped_image = warped_images[k][0].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5
target_frame = frames_tensor[k].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5

plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.imshow(frame)
plt.title('Source Frame')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(warped_image)
plt.title('Warped Image to Frame {}'.format(j))
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(target_frame)
plt.title('Target Frame {}'.format(j))
plt.axis('off')


In [ ]:
from models.utils import save_video
import os
import torch

def make_dimensions_even(tensor):
    """Pad tensor to make height and width even numbers"""
    _, h, w, c = tensor.shape
    pad_h = h % 2
    pad_w = w % 2
    
    if pad_h > 0 or pad_w > 0:
        # Pad bottom and right if needed
        tensor = torch.nn.functional.pad(tensor, (0, 0, 0, pad_w, 0, pad_h))
    
    return tensor


cond_video = (torch.cat(warped_images) + 1.0) / 2.0  # [T, 3, H, W] in [0,1]
cond_video_padded = make_dimensions_even(
    cond_video.permute(0, 2, 3, 1)
)
save_dir = '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/vkitti2_pc_warper'
os.makedirs(save_dir, exist_ok=True)

save_video(
    cond_video_padded,
    f'{save_dir}/warped_video.mp4',
    fps=10,
)
# --- save inputs for visualization ---

input_video_padded = make_dimensions_even(
    (frames_tensor.permute(0, 2, 3, 1) + 1.0) / 2.0
)

save_video(
    input_video_padded,
    f'{save_dir}/input_video.mp4',
    fps=10,
)

warped_depths_tensor = torch.cat(warped_depths)
# Apply before saving
warped_depths_padded = make_dimensions_even(
    (warped_depths_tensor.permute(0, 2, 3, 1).repeat(1, 1, 1, 3)) / warped_depths_tensor.max()
)
save_video(
    warped_depths_padded,
    f'{save_dir}/warped_depths.mp4',
    fps=10,
)

depths_padded = make_dimensions_even(
    (depths.permute(0, 2, 3, 1).repeat(1, 1, 1, 3)) / depths.max()
)
save_video(
    depths_padded,
    f'{save_dir}/input_depths.mp4',
    fps=10,
)

## Viser

In [ ]:
import viser
import numpy as np

# Start viser server
server = viser.ViserServer()
import os

# print slurm node name

node_name=os.environ.get('SLURM_NODELIST', 'localhost')

print(f'http://{node_name}:{server.get_port()}')


def add_points(
    server,
    points: np.ndarray,  # (N, 3)
    colors: np.ndarray,  # (N, 3)
    name: str,
):
    # ensure colors are in [0, 1]
    if colors.min() < -0.1:
        colors = (colors + 1.0) / 2.0
    
    server.scene.add_point_cloud(
        name=name,
        points=points,
        colors=colors,
        point_size=0.01
    )

def add_camera(
    server,
    pose: np.ndarray,  # (4, 4)
    name: str,
    color: tuple = (0.2, 0.8, 0.2),
):
    pose = np.linalg.inv(pose)
    
    position = pose[:3, 3]
    rotation_matrix = pose[:3, :3]
    
    # Convert rotation to quaternion
    wxyz = viser.transforms.SO3.from_matrix(rotation_matrix).wxyz
    
    server.scene.add_camera_frustum(
        name,
        fov=60, aspect=4/3, scale=0.1,
        position=position, wxyz=wxyz,
        color=color
    )
    
# Add this after creating your server and adding point clouds
@server.on_client_connect
def _(client: viser.ClientHandle) -> None:
    yaw_slider = client.gui.add_slider("Camera Yaw", min=-180, max=180, step=1, initial_value=0)
    
    @yaw_slider.on_update
    def _(_):
        angle_rad = np.deg2rad(yaw_slider.value)
        radius = 8.0
        
        position = np.array([
            radius * np.sin(angle_rad),
            0,  # height
            radius * np.cos(angle_rad)
        ])
        
        client.camera.position = position
        client.camera.look_at = np.array([0, 0, 0])
        client.camera.up_direction = np.array([0, -1, 0])


In [ ]:
poses_tensor

In [ ]:
pc_list[0].shape

In [ ]:
server.scene.reset()

# add original point cloud #0, and all cameras from first segment

j_list = [10]

for j in j_list:    
    add_points(
        server,
        pc_list[j].cpu().numpy()[::10,:],
        color_list[j].cpu().numpy()[::10,:],
        name=f'input_pc_{j}'
    )
    
for j in range(frames_tensor.shape[0]):
    add_camera(
        server,
        torch.inverse(poses_tensor[j]).cpu().numpy(),
        name=f'input_cam_{j:02d}',
        color=(0.2, 0.8, 0.2)
    )
   

In [ ]:
depths.min(), depths.max()

In [ ]:
pc_list[10]

In [95]:
sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/06_10_25_vggt')
from parsing import get_parser
import utils_autoregressive as utils_ar
from datetime import datetime
import os
import copy


sys.argv = [
    "",
    "--video_path", "/home/azhuravl/nobackup/DAVIS_testing/trainval/vkitti2.mp4",
    "--n_splits", "4",
    "--overlap_frames", "0",
    "--radius", "0",
    "--mode", "gradual",
]

parser = get_parser()
opts_base = parser.parse_args()

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
video_basename = os.path.splitext(os.path.basename(opts_base.video_path))[0]

# Setup
opts_base.weight_dtype = torch.bfloat16
opts_base.exp_name = f"{video_basename}_{timestamp}_autoregressive"
opts_base.save_dir = os.path.join(opts_base.out_dir, opts_base.exp_name)

# Create TrajCrafterVisualization instance for autoregressive generation
radius = opts_base.radius

variants = [
    ("right_90", [0, 90, radius, 0, 0]),
]

pose = [90, 0, 0, 0, 1]
name = f"{pose[0]}_{pose[1]}_{pose[2]}_{pose[3]}_{pose[4]}"

opts = copy.deepcopy(opts_base)
opts.exp_name = f"{video_basename}_{timestamp}_{name}_auto_s{opts_base.n_splits}"
opts.save_dir = os.path.join(opts.out_dir, opts.exp_name)
opts.camera = "target"
opts.target_pose = pose
opts.traj_txt = 'test/trajs/loop2.txt'

# Make directories
os.makedirs(opts.save_dir, exist_ok=True)

In [96]:
trajcrafter = utils_ar.TrajCrafterAutoregressive(opts)

In [22]:
import numpy as np

# frames_tensor = (
    # torch.from_numpy(frames_np).permute(0, 3, 1, 2).to(opts.device) * 2.0 - 1.0
  # 49 576 1024 3 -> 49 3 576 1024, [-1,1]
# reverse this to get frames in numpy
frames_np = ((frames_tensor.cpu().permute(0, 2, 3, 1).numpy() + 1.0) / 2.0).astype(np.float32)

trajcrafter.prompt = trajcrafter.get_caption(opts, frames_np[opts.video_length // 2])
print(trajcrafter.prompt)

In [23]:
_, segment_dir = utils_ar.sample_diffusion(
    trajcrafter,
    frames_tensor[10:],
    warped_images,
    frames_tensor[:10],
    masks,
    opts,
)

## TartanAir

In [ ]:
import importlib
importlib.reload(video_datasets)

In [ ]:
train_sequences = [
    'abandonedfactory/Easy/P001',
    # 'abandonedfactory/Easy/P005', 
    # 'office/Easy/P001',
    # 'office/Easy/P002',
    # 'office2/Easy/P001'
]

dataset_tartanair = video_datasets.TartanAirDataset(
        aug_params=None,
        root="/home/azhuravl/scratch/tartanair",
        split="train",
        sample_len=59,
        only_first_n_samples=-1,
        sampling_stride=3,          # Starting frame stride (default 3)
        min_temporal_step=1,        # Minimum temporal step (default 1)  
        max_temporal_step=2,        # Maximum temporal step (default 6)
        train_sequences=train_sequences
    )

In [ ]:
data_0 = dataset_tartanair[10]

In [ ]:
data_0

In [ ]:
data_0 = dataset_tartanair[0]
data_1 = dataset_tartanair[1]


In [ ]:
# TODO: fix TartanAir cameras